In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision import datasets,models

import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
path='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Gender'

In [3]:
train_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])

In [4]:
train_data=datasets.ImageFolder(os.path.join(path,'train'),transform=train_transform)
test_data=datasets.ImageFolder(os.path.join(path,'test'),transform=test_transform)

In [5]:
len(train_data),len(test_data)

(47009, 11649)

In [6]:
class_names=train_data.classes
class_names

['female', 'male']

In [8]:
torch.manual_seed(101)
train_loader=DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=32,shuffle=True)

In [9]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    model.train()
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    model.eval() 
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

from tqdm.auto import tqdm

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int):
    
    results={'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}
    
    for i in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
        test_loss,test_acc=test_step(model=model,
                                     dataloader=test_dataloader,
                                     loss_fn=loss_fn)
        print(f'Epoch {i} | Train loss: {train_loss:.4f},Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f},Test acc: {test_acc:.4f}')
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    return results

In [10]:
class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(3,6,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(6,16,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),
            nn.Linear(54*54*16,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,len(class_names))
        )

    def forward(self,X):
        return F.log_softmax(self.conv(X))
    

In [11]:
torch.manual_seed(101)
model1=Model()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model1.parameters(),lr=0.001)

In [12]:
torch.manual_seed(101)

n=10
model_results=train(
    model=model1,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 | Train loss: 0.3007,Train acc: 0.8693 | Test loss: 0.2310,Test acc: 0.8996
Epoch 1 | Train loss: 0.2159,Train acc: 0.9149 | Test loss: 0.2081,Test acc: 0.9147
Epoch 2 | Train loss: 0.1979,Train acc: 0.9234 | Test loss: 0.1585,Test acc: 0.9381
Epoch 3 | Train loss: 0.1854,Train acc: 0.9284 | Test loss: 0.1663,Test acc: 0.9357
Epoch 4 | Train loss: 0.1750,Train acc: 0.9319 | Test loss: 0.1532,Test acc: 0.9402
Epoch 5 | Train loss: 0.1664,Train acc: 0.9366 | Test loss: 0.1413,Test acc: 0.9447
Epoch 6 | Train loss: 0.1590,Train acc: 0.9396 | Test loss: 0.1432,Test acc: 0.9468
Epoch 7 | Train loss: 0.1477,Train acc: 0.9446 | Test loss: 0.1597,Test acc: 0.9398
Epoch 8 | Train loss: 0.1459,Train acc: 0.9455 | Test loss: 0.1297,Test acc: 0.9518
Epoch 9 | Train loss: 0.1365,Train acc: 0.9493 | Test loss: 0.1296,Test acc: 0.9510


In [13]:
index=list(range(11649))

c=0
for i in index:
    model1.eval()
    with torch.no_grad():
        pred=model1(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Number of correct: 11077/11649 | Accuracy: 95.08970727101038
